### Exploratory Data Analysis - USDA Food Access Research Atlas (Tennessee data only)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
tn_food_access_df = pd.read_csv('../data/tn_food_access.csv')

In [ ]:
tn_food_access_df = tn_food_access_df.drop('State', axis=1)

In [ ]:
tn_food_access_df = tn_food_access_df.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# tn_food_access_df_fillna.to_csv('tn_food_access_nan.csv')

Getting a general sense of the data ...

In [ ]:
tn_food_access_df.head()

In [ ]:
tn_food_access_df['CT_Clean'].info()

993 unique entries for census tract, yet 1,489 total? Need to look into this as a top priority.

In [ ]:
tn_food_access_df['CT_Clean'].nunique()

In [ ]:
# https://saturncloud.io/blog/how-to-check-for-duplicate-values-in-pandas-dataframe-column/#:~:text=To%20check%20for%20duplicate%20values%20in%20a%20Pandas%20DataFrame%20column,is%20a%20duplicate%20or%20not.
# duplicate_values = df['Product'].duplicated()
# print(duplicate_values)

duplicate_CTs = tn_food_access_df['CT_Clean'].duplicated()
duplicate_CTs_to_csv = pd.DataFrame(duplicate_CTs)
# duplicate_CTs_to_csv.to_csv('duplicate_CTs.csv')


ISSUE FOR REVIEW: converting the census tract numbers to match the census reporter data did not work and created duplicate census tract IDs in about 1/3 of the data. I will probably need to go back and change some things but need some help troubleshooting and figuring out where to start/what's needed.

In [ ]:
tn_food_access_df['CT_Old'].info()

In [ ]:
tn_food_access_df['CT_Old'].nunique()

Pulling average, median, maximum and minimum population values. 

In [ ]:
tn_food_access_df['Pop2010'].mean()

The average exceeds median population in Tennessee by about 200 people.

In [ ]:
tn_food_access_df['Pop2010'].median()

Maximum population in any census tract is 21,763 people in CT 501.01 Williamson County.

Williamson County is just south of Nashville and includes Franklin, TN.

In [ ]:
# https://stackoverflow.com/questions/71828530/printing-the-whole-row-of-my-data-from-a-max-value-in-a-column
tn_food_access_df[tn_food_access_df['Pop2010'].iloc[:]==tn_food_access_df['Pop2010'].iloc[:].max()]

Pulling minimum population, I'm seeing two lines that appear to have the same census tract number. 

Sevier County and Hamilton County are hours apart by car, so assuming this is some kind of database error.

* Sevier County is east and slightly south of Knoxville and includes Gatlinburg.

* Hamilton County is in southeast Tennessee and includes Chattanooga.

In [ ]:
tn_food_access_df[tn_food_access_df['Pop2010'].iloc[:]==tn_food_access_df['Pop2010'].iloc[:].min()]

In [ ]:
# https://thinkingneuron.com/exporting-data-out-of-python/
# # Creating a DataFrame
# DataSample= [[10,'value1'], 
#              [20,'value2'], 
#              [30,'value3']]
# SimpleDataFrame=pd.DataFrame(data=DataSample, columns=['Col1','Col2'])
# print(SimpleDataFrame)
 
# # Exporting data frame to a csv/Excel file
# # Many other options are available which can be seen using dot tab option
 
# # Exporting data as a csv file
# SimpleDataFrame.to_csv('C:/Users/fhashmi/SimpleDataFrame.csv')
 
# # Exporting data as a excel file
# SimpleDataFrame.to_excel('C:/Users/fhashmi/SimpleDataFrameExcel.xlsx')

Saving a CSV to the data folder with population by county. (Also commenting it out so I don't get a new one each time I run the notebook.)

In [ ]:
# data.groupby(['month','network_type'])['date'].count()
pop_by_county = tn_food_access_df.groupby(['County'])['Pop2010'].mean()
pop_by_county_to_csv = pd.DataFrame(pop_by_county)
# pop_by_county_to_csv.to_csv('pop_by_county.csv')

Maybe a few graphs to get started and help guide where to go.

In [ ]:
# https://stackoverflow.com/questions/19442060/matplotlib-pyplot-titlestring-returns-error
# this was a fix for the title error I was getting ("'str' object is not callable")
import matplotlib.pyplot as plt
from importlib import reload
plt=reload(plt)

In [ ]:
sns.scatterplot(data=tn_food_access_df, x='Pop2010', y='TractLOWI')
plt.title('Total Population and Low Income Population')

The above scatterplot isn't groundbreaking, but it does show that as total population increases, low income population does too. Makes sense, though the correlation seems like it's higher than I would have expected. Some more work here couldn't hurt. Maybe this? https://forecastegy.com/posts/correlation-between-two-time-series-python/

In [ ]:
sns.scatterplot(data=tn_food_access_df, x='Pop2010', y='TractSNAP')
plt.title('Total Population and Housing Units With SNAP')

In [ ]:
sns.scatterplot(data=tn_food_access_df, x='Pop2010', y='TractHUNV')
plt.title('Total Population and Housing Units Without Vehicle Access')

Going to have to rethink how I analyze the places with just a flag, like the LILATracts_1and20

In [ ]:
sns.scatterplot(data=tn_food_access_df, x='Pop2010', y='LILATracts_1And20')
plt.title('Total Population and Low Access/Low Income at 1 Mile Urban and 20 Miles Rural')